# 모델링 구현
# 랜덤 포레스트 분류 모델

### 의사결정 트리
- 자료구조 중 하나인 트리 구조와 같은 형태로 이뤄진 알고리즘

### 랜덤 포레스트 
- 여러개의 의사결정 트리의 결과값을 평균낸 것을 사용

## CountVectorizer를 활용한 벡터화

- 랜덤 포레스트 모델에서는 CountVectorizer를 사용해 모델의 입력값을 만든다.
- 전처리한 텍스트 데이터를 입력으로 사용

In [12]:
import pandas as pd
import numpy as np
import os

DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
y = np.array(train_data['sentiment'])

- 모델에 적용하기 위한 특징 추출 방법으로 CountVectorizer를 이용

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', max_features=5000)

train_data_features = vectorizer.fit_transform(reviews)

- analyzer
    - 분석 단위를 하나의 단어로 지정

- max_features
    - 각 벡터의 최대 길이를 설정

In [6]:
train_data_features

<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

## 학습 데이터와 검증 데이터 분리

In [7]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2
RANDOM_SEED = 42

train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

## 모델 구현 및 학습

In [8]:
from sklearn.ensemble import RandomForestClassifier 

forest = RandomForestClassifier(n_estimators=100)

forest.fit(train_input, train_label)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## 검증 데이터셋으로 성능 평가

In [9]:
print(f"Accuracy: {forest.score(eval_input, eval_label)}")

Accuracy: 0.8454


## Kaggle 데이터 제출


In [15]:
TEST_CLEAN_DATA = 'test_clean.csv'
DATA_OUT_PATH = './data_out/'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_reviews = list(test_data['review'])
ids = list(test_data['id'])

In [16]:
test_data_features = vectorizer.transform(test_reviews)

In [17]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

result = forest.predict(test_data_features)

output = pd.DataFrame(data = {'id': ids, 'sentiment': result})

output.to_csv(DATA_OUT_PATH + 'bag_of_word_model.csv', index=False, quoting=3)